# Helgeson, Delany, Nesbitt and Bird Standard State Code Generator 

Helgeson HC, Delany JM, Nesbitt HW, and Bird DK (1978) Summary and critique of the thermodynamic properties of rock-forming minerals, American Journal of Sciences, 278-A, 229pp

This notebook generates code for all minerals in Helgeson et al. (1992) except quartz, which is treated in [another notebook](HDNB-Quartz.ipynb) as a special case. This notebook also generates code for gas phases.  It does not generate code for aqueous species, which are treated in a [separate notebook](HKF.ipynb).

Four cases are tested:
- Minerals without phase transitions
- Minerals with one phase transition
- Minerals with two phase transitions
- Minerals with three phase transitions
- A gas phase

Required system packages and initialization ...

In [ ]:
import pandas as pd
import numpy as np
import sympy as sym
import pickle
import math
sym.init_printing()

Required ENKI packages

In [ ]:
from thermoengine import coder

# Formulation
There are three classes of terms:
1. Terms that apply over the whole of $T$-, $P$-space, $T_r \le T$, $P_r \le P$
2. Terms that apply over a specified range of $T$-, $P$-space, $(T_{r_\lambda},P_{r_\lambda}) \le (T,P) \le (T_\lambda,P_\lambda)$
3. Terms that apply to a specific $T_t$ and $P_t$ and higher $T$, $P$, $T_t \le T$, $P_t \le P$

### Helgeson et al. (1978) model structure
The isobaric heat capacity is represented by a Maier-Kelley type expression, ${C_P} = a + bT + \frac{c}{{{T^2}}}$. A phase transition is represented by a finite change in enthalpy and volume (and since the free energy change at the transition is zero, a dependent change in entropy) at some $T_t$. Above the transition temperature, which can be pressure dependent, a new heat capacity expression is used.  There can be up to three transitions for a single phase.  Gasses and most minerals do not undergo phase transition.  More than one phase transition is unusual.

Create a model class for the Gibbs free energy

In [ ]:
model = coder.StdStateModel()

Retrieve sympy symbols for model variables and reference conditions

In [ ]:
T = model.get_symbol_for_t()
P = model.get_symbol_for_p()
Tr = model.get_symbol_for_tr()
Pr = model.get_symbol_for_pr()

- The isobaric heat capacity terms parameterized as: $C_P = a_0 + b_0 T + c_0 / T^2$
- Third law entropy: $ S_{Tr,Pr} $
- Enthalpy of formation from the elements, $ \Delta H_{Tr,Pr} $
- $T_t$ is the phase transition temperature
- $\Delta H_t$ is the transition enthalpy at $T_t$
- $\Delta V_t$ is the transition volume at $T_t$
- ${\left. {\frac{{dP}}{{dT}}} \right|_t}$ is the Clapyron slope of the phase transition at $T_t$.  Here the inverse is used in order to more easily parameterize the model.

In [ ]:
a0,b0,c0,a1,b1,c1,a2,b2,c2,a3,b3,c3 = sym.symbols('a0 b0 c0 a1 b1 c1 a2 b2 c2 a3 b3 c3')
Tt1,Tt2,Tt3 = sym.symbols('Tt1 Tt2 Tt3')
DeltaH1,DeltaH2,DeltaH3 = sym.symbols('DeltaH1 DeltaH2 DeltaH3')
DeltaV1,DeltaV2,DeltaV3 = sym.symbols('DeltaV1 DeltaV2 DeltaV3')
dPdT1,dPdT2,dPdT3 = sym.symbols('dPdT1 dPdT2 dPdT3')
STrPr,GTrPr,VTrPr = sym.symbols('S_TrPr G_TrPr V_TrPr')

Contributions applicable over the whole of *T*, *P* space ...

In [ ]:
g = GTrPr - STrPr*(T-Tr) + VTrPr*(P-Pr)
params = [('G_TrPr','J/m',GTrPr), ('S_TrPr','J/K-m',STrPr), ('V_TrPr', 'J/bar-m', VTrPr)]
model.add_expression_to_model(g, params)

Heat capacity contribution up to the first transition ...

In [ ]:
Cp = a0+b0*T+c0/T**2
g = sym.integrate(Cp,(T,Tr,T)) - T*sym.integrate(Cp/T,(T,Tr,T))
params = [('a0','J/K-m',a0), ('b0','J/K^2-m',b0), ('c0','J-K/m',c0), ('Tt1','K',Tt1)]
model.add_expression_to_model(g, params, exp_type='restricted', lower_limits=(None,None), upper_limits=(Tt1,None))
gAtTt1 = g.subs(T, Tt1)

First order phase transition contribution at the first transition ...

In [ ]:
g = -(T-Tt1)*DeltaH1/Tt1
params = [('DeltaH1','J/m',DeltaH1)]
model.add_expression_to_model(g, params, exp_type='restricted', lower_limits=(Tt1,None), upper_limits=(None,None))
P1 = Pr + dPdT1*(T-Tt1)
g = (P-Pr)*DeltaV1
params = [('DeltaV1','J/bar-m',DeltaV1), ('dPdT1','bar/K',dPdT1)]
model.add_expression_to_model(g, params, exp_type='restricted', lower_limits=(Tt1,None), upper_limits=(None,P1)) 

Heat capacity contribution above the first transition and up to the second transition ...

In [ ]:
Cp = a1+b1*T+c1/T**2
g = sym.integrate(Cp,(T,Tt1,T)) - T*sym.integrate(Cp/T,(T,Tt1,T))
params = [('a1','J/K-m',a1), ('b1','J/K^2-m',b1), ('c1','J-K/m',c1), ('Tt2','K',Tt2)]
model.add_expression_to_model(g + gAtTt1, params, exp_type='restricted', lower_limits=(Tt1,None), upper_limits=(Tt2,None))
gAtTt2 = g.subs(T, Tt2)

First order phase transition contribution at the second transition ...

In [ ]:
g = -(T-Tt2)*DeltaH2/Tt2
params = [('DeltaH2','J/m',DeltaH2)]
model.add_expression_to_model(g, params, exp_type='restricted', lower_limits=(Tt2,None), upper_limits=(None,None))
P2 = Pr + dPdT2*(T-Tt2)
g = (P-Pr)*DeltaV2
params = [('DeltaV2','J/bar-m',DeltaV2), ('dPdT2','bar/K',dPdT2)]
model.add_expression_to_model(g, params, exp_type='restricted', lower_limits=(Tt2, None), upper_limits=(None, P2))

Heat capacity contribution above the second transition and up to the third transition ...

In [ ]:
Cp = a2+b2*T+c2/T**2
g = sym.integrate(Cp,(T,Tt2,T)) - T*sym.integrate(Cp/T,(T,Tt2,T))
params = [('a2','J/K-m',a2), ('b2','J/K^2-m',b2), ('c2','J-K/m',c2), ('Tt3','K',Tt3)]
model.add_expression_to_model(g + gAtTt1 + gAtTt2, params, exp_type='restricted', lower_limits=(Tt2,None), upper_limits=(Tt3,None))
gAtTt3 = g.subs(T, Tt3)

First order phase transition contribution at the third transition ...

In [ ]:
g = -(T-Tt3)*DeltaH3/Tt3
params = [('DeltaH3','J/m',DeltaH3)]
model.add_expression_to_model(g, params, exp_type='restricted', lower_limits=(Tt3,None), upper_limits=(None,None))
P3 = Pr + dPdT3*(T-Tt3)
g = (P-Pr)*DeltaV3
params = [('DeltaV3','J/bar-m',DeltaV3), ('dPdT3','bar/K',dPdT3)]
model.add_expression_to_model(g, params, exp_type='restricted', lower_limits=(Tt3,None), upper_limits=(None,P3))

Heat capacity contribution above the third transition and up to the third transition ...

In [ ]:
Cp = a3+b3*T+c3/T**2
g = sym.integrate(Cp,(T,Tt3,T)) - T*sym.integrate(Cp/T,(T,Tt3,T))
params = [('a3','J/K-m',a3), ('b3','J/K^2-m',b3), ('c3','J-K/m',c3)]
model.add_expression_to_model(g + gAtTt1 + gAtTt2 + gAtTt3, params, exp_type='restricted', lower_limits=(Tt3,None), 
                              upper_limits=(None,None))

# Code Print the Model, compile the code and link a Python module
Name the model class

In [ ]:
model.set_module_name('hdnb')

## Retrieve the Helgeson et al. (1978) database ...
Load a SLOP file, which consists of dictionaries of Pandas dataframes, and extract the parameters of a phase from the one of the dataframes.  

In [ ]:
with open('slop16_v3_1.dict', 'rb') as pickle_file:
    slop_d = pickle.load(pickle_file)

Contents of the SLOP dictionary ...

In [ ]:
for key in slop_d.keys():
    print ('key:', key)

### Select select which class of minerals and gasses to test ...

In [ ]:
test_min_no_transitions = True
test_min_one_transition = False
test_min_two_transitions = False
test_min_three_transitions = False
test_gasses = False
test_case_no_transitions = 'Calcite'
test_case_one_transition = 'Ferrosilite' # 'Ferrosilite' 'Cristobalite' 'Albite'
test_case_two_transitions = 'Enstatite'
test_case_three_transitions = 'Iron'
test_case_gasses = 'Hydrogen'

In [ ]:
ref_properties = {
    'Calcite':{'T':1000.0, 'P':10000.0, 'G':-288869.4, 'H':-261265.3, 'S':52.88013, 'V':36.934, 'Cp':29.6},
    'Ferrosilite':{'T':1000.0, 'P':10000.0, 'G':-288053.7, 'H':-258018.7, 'S':54.55396, 'V':33.008, 'Cp':32.175},
    'Enstatite':{'T':1600.0, 'P':1000.0,  'G':-403698.9, 'H':-332978.2, 'S':60.19175, 'V':32.386, 'Cp':29.26},
    'Iron':{'T':2000.0, 'P':1000.0,  'G':0.0,       'H':16021.59,  'S':22.98924, 'V':7.092,  'Cp':9.94},
    'Hydrogen':{'T':2000.0, 'P':1000.0,  'G':-68105.93, 'H':12655.64,  'S':45.03699, 'V':0.0,    'Cp':8.083},
    'Cristobalite':{'T':1000.0, 'P':10000.0, 'G':-212158.5, 'H':-199867.5, 'S':28.24339, 'V':25.74, 'Cp':16.711},
    'Albite':{'T':1000.0, 'P':10000.0, 'G':-928580.2, 'H':-866950.8, 'S':129.799, 'V':100.25, 'Cp':80.4186}
    #'Albite':{'T':450.0, 'P':10000.0, 'G':-871655.2, 'H':-907261.3, 'S':72.28305, 'V':100.25, 'Cp':60.54265}
}

### Retrieve test phase properties from appropriate dataframe ...

In [ ]:
if test_min_no_transitions:
    min_df = slop_d.get("mineral_no_transitions")
    min_name = test_case_no_transitions
elif test_min_one_transition:
    min_df = slop_d.get("mineral_one_transition")
    min_name = test_case_one_transition
elif test_min_two_transitions:
    min_df = slop_d.get("mineral_two_transitions")
    min_name = test_case_two_transitions
elif test_min_three_transitions:
    min_df = slop_d.get("mineral_three_transitions")
    min_name = test_case_three_transitions
elif test_gasses:
    min_df = slop_d.get("gasses")
    min_name = test_case_gasses

labels = min_df.columns.tolist()
if test_gasses:
    content = min_df.loc[min_df['Struct_formula'] == min_name].get_values().tolist()[0]
else:
    content = min_df.loc[min_df['Name'] == min_name].get_values().tolist()[0]
phase = dict(zip(labels,content))
for key in phase.keys():
    try:
        if np.isnan(phase[key]):
            phase[key] = '0.0'
    except (TypeError):
        pass
    print ("{0:<25.25s} {1:20.20s}".format(key, phase[key]))

Load parameters into a dictionary for code creation ...

In [ ]:
calToJoules = 4.184
param_dict = {
    'S_TrPr':float(phase['S (cal/K-m)'])*calToJoules,
    'G_TrPr':float(phase['deltaG (cal/m)'])*calToJoules,
    'V_TrPr':float(phase['V (cc/m)'])/10.0, # J/bar-m
    'a0':float(phase['a (cal/K-m)'])*calToJoules,
    'b0':float(phase['b (10^3 cal/K^2-m)'])*calToJoules/1000.0,
    'c0':float(phase['c (10^-5 cal-K/m)'])*calToJoules*100000.0,
    'Tt1':max(float(phase['Tmax (K)']), 2273.15),
    'DeltaH1':0.0,
    'DeltaV1':0.0,
    'dPdT1':0.0,
    'a1':0.0,
    'b1':0.0,
    'c1':0.0,
    'Tt2':max(float(phase['Tmax (K)']), 2273.15),
    'DeltaH2':0.0,
    'DeltaV2':0.0,
    'dPdT2':0.0,
    'a2':0.0,
    'b2':0.0,
    'c2':0.0,
    'Tt3':max(float(phase['Tmax (K)']), 2273.15),
    'DeltaH3':0.0,
    'DeltaV3':0.0,
    'dPdT3':0.0,
    'a3':0.0,
    'b3':0.0,
    'c3':0.0,
    'T_r':298.15,
    'P_r':1.0
}
if test_min_one_transition or test_min_two_transitions or test_min_three_transitions:
    param_dict['Tt1']     = float(phase['Tt1 (K)'])
    param_dict['DeltaH1'] = float(phase['DeltaHt1 (cal/m)'])*calToJoules
    param_dict['DeltaV1'] = float(phase['deltaVt1 (cc/m)'])/10.0 # J/bar-m
    param_dict['dPdT1']   = float(phase['dPdTt1 (bar/K)'])
    param_dict['a1']      = float(phase['at1 (cal/K-m)'])*calToJoules
    param_dict['b1']      = float(phase['bt1 (10^3 cal/K^2-m)'])*calToJoules/1000.0
    param_dict['c1']      = float(phase['ct1 (10^-5 cal-K/m)'])*calToJoules*100000.0
    param_dict['Tt2']     = max(float(phase['Tmax (K)']), 2273.15)
if test_min_two_transitions or test_min_three_transitions:
    param_dict['Tt2']     = float(phase['Tt2 (K)'])
    param_dict['DeltaH2'] = float(phase['DeltaHt2 (cal/m)'])*calToJoules
    param_dict['DeltaV2'] = float(phase['deltaVt2 (cc/m)'])/10.0 # J/bar-m
    param_dict['dPdT2']   = float(phase['dPdTt2 (bar/K)'])
    param_dict['a2']      = float(phase['at2 (cal/K-m)'])*calToJoules
    param_dict['b2']      = float(phase['bt2 (10^3 cal/K^2-m)'])*calToJoules/1000.0
    param_dict['c2']      = float(phase['ct2 (10^-5 cal-K/m)'])*calToJoules*100000.0
    param_dict['Tt3']     = max(float(phase['Tmax (K)']), 2273.15)
if test_min_three_transitions:
    param_dict['Tt3']     = float(phase['Tt3 (K)'])
    param_dict['DeltaH3'] = float(phase['DeltaHt3 (cal/m)'])*calToJoules
    param_dict['DeltaV3'] = float(phase['deltaVt3 (cc/m)'])/10.0 # J/bar-m
    param_dict['dPdT3']   = float(phase['dPdTt3 (bar/K)'])
    param_dict['a3']      = float(phase['at3 (cal/K-m)'])*calToJoules
    param_dict['b3']      = float(phase['bt3 (10^3 cal/K^2-m)'])*calToJoules/1000.0
    param_dict['c3']      = float(phase['ct3 (10^-5 cal-K/m)'])*calToJoules*100000.0

# normally the next entry would be set to min_name, but here it is set to 
# generic to allow the testing functions to be applicable to all test cases 
phase_name = 'Generic' 
formula = 'Na(1)Al(1)Si(3)O(8)' if min_name == 'Albite' else phase['Formula']

Make a working sub-directory and move down into the directory.  This is done so that generated files will not clash between alternate model configurations.

In [ ]:
model_working_dir = "hdnb"
!mkdir -p {model_working_dir}
%cd {model_working_dir}

Note that the call to
```
model.create_code_module(phase=phase_name, formula=formula, params=param_dict)
```
generates fast code with unmodifiable model parameters and "calibration-" related functions.  The call to:
```
model.create_code_module(phase=phase_name, formula=formula, params=param_dict, module_type='calib')
```
generates code suitable for model parameter calibration.  
model_type is "fast" or "calib"

In [ ]:
model_type = "fast"

In [ ]:
result = model.create_code_module(phase=phase_name, formula=formula, params=param_dict, module_type=model_type)

## Import the new module and test the model

In [ ]:
import hdnb
%cd ..

Evaluate functions at temperature (K) and pressure (bars)

In [ ]:
if test_min_no_transitions:
    case_name = test_case_no_transitions
elif test_min_one_transition:
    case_name = test_case_one_transition
elif test_min_two_transitions:
    case_name = test_case_two_transitions
elif test_min_three_transitions:
    case_name = test_case_three_transitions
elif test_gasses:
    case_name = test_case_gasses
t = ref_properties[case_name]['T']
p = ref_properties[case_name]['P']

### Available in both "Fast" and "Calib" code versions 
Execute the "fast" or "calibration" code metadata retrieval functions:

In [ ]:
try:
    print(hdnb.cy_Generic_hdnb_identifier())
    print(hdnb.cy_Generic_hdnb_name())
    print(hdnb.cy_Generic_hdnb_formula())
    print(hdnb.cy_Generic_hdnb_mw())
    print(hdnb.cy_Generic_hdnb_elements())
except AttributeError:
    pass
try:
    print(hdnb.cy_Generic_hdnb_calib_identifier())
    print(hdnb.cy_Generic_hdnb_calib_name())
    print(hdnb.cy_Generic_hdnb_calib_formula())
    print(hdnb.cy_Generic_hdnb_calib_mw())
    print(hdnb.cy_Generic_hdnb_calib_elements())
except AttributeError:
    pass

Execute the standard thermodynamic property retrieval functions:

In [ ]:
fmt = "{0:<10.10s} {1:13.6e} {2:<10.10s}"
fmc = "{0:<10.10s} {1:13.6e} {2:13.6e} {3:10.3f} % {4:<10.10s}"
try:
    print(fmc.format('G', hdnb.cy_Generic_hdnb_g(t,p), 
                     ref_properties[case_name]['G']*4.184, 
                     100.0*(hdnb.cy_Generic_hdnb_g(t,p)-ref_properties[case_name]['G']*4.184)/
                     ((ref_properties[case_name]['G']*4.184) if ref_properties[case_name]['G'] != 0 else 1.0), 
                     'J/m'))
    print(fmc.format('dGdT', hdnb.cy_Generic_hdnb_dgdt(t,p), 
                     -ref_properties[case_name]['S']*4.184, 
                     100.0*(hdnb.cy_Generic_hdnb_dgdt(t,p)+ref_properties[case_name]['S']*4.184)/(-ref_properties[case_name]['S']*4.184), 
                     'J/K-m'))
    print(fmc.format('dGdP', hdnb.cy_Generic_hdnb_dgdp(t,p), 
                     ref_properties[case_name]['V']/10.0, 
                     100.0*(hdnb.cy_Generic_hdnb_dgdp(t,p)-ref_properties[case_name]['V']/10.0)/
                     ((ref_properties[case_name]['V']/10.0) if ref_properties[case_name]['V'] != 0.0 else 1.0), 
                     'J/bar-m'))
    print(fmc.format('d2GdT2', hdnb.cy_Generic_hdnb_d2gdt2(t,p), 
                     -ref_properties[case_name]['Cp']*4.184/t, 
                     100.0*(hdnb.cy_Generic_hdnb_d2gdt2(t,p)+ref_properties[case_name]['Cp']*4.184/t)/
                     (ref_properties[case_name]['Cp']*4.184/t), 
                     'J/K^2-m'))
    print(fmt.format('d2GdTdP', hdnb.cy_Generic_hdnb_d2gdtdp(t,p), 'J/K-bar-m'))
    print(fmt.format('d2GdP2', hdnb.cy_Generic_hdnb_d2gdp2(t,p), 'J/bar^2-m'))
    print(fmt.format('d3GdT3', hdnb.cy_Generic_hdnb_d3gdt3(t,p), 'J/K^3-m'))
    print(fmt.format('d3GdT2dP', hdnb.cy_Generic_hdnb_d3gdt2dp(t,p), 'J/K^2-bar-m'))
    print(fmt.format('d3GdTdP2', hdnb.cy_Generic_hdnb_d3gdtdp2(t,p), 'J/K-bar^2-m'))
    print(fmt.format('d3GdP3', hdnb.cy_Generic_hdnb_d3gdp3(t,p), 'J/bar^3-m'))
    print(fmc.format('S', hdnb.cy_Generic_hdnb_s(t,p), 
                     ref_properties[case_name]['S']*4.184, 
                     100.0*(hdnb.cy_Generic_hdnb_s(t,p)-ref_properties[case_name]['S']*4.184)/(ref_properties[case_name]['S']*4.184), 
                     'J/K-m'))
    print(fmc.format('V', hdnb.cy_Generic_hdnb_v(t,p), 
                     ref_properties[case_name]['V']/10.0, 
                     100.0*(hdnb.cy_Generic_hdnb_dgdp(t,p)-ref_properties[case_name]['V']/10.0)/
                     ((ref_properties[case_name]['V']/10.0) if ref_properties[case_name]['V'] != 0.0 else 1.0), 
                     'J/bar-m'))
    print(fmt.format('Cv', hdnb.cy_Generic_hdnb_cv(t,p), 'J/K-m'))
    print(fmc.format('Cp', hdnb.cy_Generic_hdnb_cp(t,p), 
                     ref_properties[case_name]['Cp']*4.184, 
                     100.0*(hdnb.cy_Generic_hdnb_cp(t,p)-ref_properties[case_name]['Cp']*4.184)/(ref_properties[case_name]['Cp']*4.184), 
                     'J/K-m'))
    print(fmt.format('dCpdT', hdnb.cy_Generic_hdnb_dcpdt(t,p), 'J/K^2-m'))
    print(fmt.format('alpha', hdnb.cy_Generic_hdnb_alpha(t,p), '1/K'))
    print(fmt.format('beta', hdnb.cy_Generic_hdnb_beta(t,p), '1/bar'))
    print(fmt.format('K', hdnb.cy_Generic_hdnb_K(t,p), 'bar'))
    print(fmt.format('Kp', hdnb.cy_Generic_hdnb_Kp(t,p), ''))
except AttributeError:
    pass
try:
    print(fmt.format('G', hdnb.cy_Generic_hdnb_calib_g(t,p), 'J/m'))
    print(fmt.format('dGdT', hdnb.cy_Generic_hdnb_calib_dgdt(t,p), 'J/K-m'))
    print(fmt.format('dGdP', hdnb.cy_Generic_hdnb_calib_dgdp(t,p), 'J/bar-m'))
    print(fmt.format('d2GdT2', hdnb.cy_Generic_hdnb_calib_d2gdt2(t,p), 'J/K^2-m'))
    print(fmt.format('d2GdTdP', hdnb.cy_Generic_hdnb_calib_d2gdtdp(t,p), 'J/K-bar-m'))
    print(fmt.format('d2GdP2', hdnb.cy_Generic_hdnb_calib_d2gdp2(t,p), 'J/bar^2-m'))
    print(fmt.format('d3GdT3', hdnb.cy_Generic_hdnb_calib_d3gdt3(t,p), 'J/K^3-m'))
    print(fmt.format('d3GdT2dP', hdnb.cy_Generic_hdnb_calib_d3gdt2dp(t,p), 'J/K^2-bar-m'))
    print(fmt.format('d3GdTdP2', hdnb.cy_Generic_hdnb_calib_d3gdtdp2(t,p), 'J/K-bar^2-m'))
    print(fmt.format('d3GdP3', hdnb.cy_Generic_hdnb_calib_d3gdp3(t,p), 'J/bar^3-m'))
    print(fmt.format('S', hdnb.cy_Generic_hdnb_calib_s(t,p), 'J/K-m'))
    print(fmt.format('V', hdnb.cy_Generic_hdnb_calib_v(t,p), 'J/bar-m'))
    print(fmt.format('Cv', hdnb.cy_Generic_hdnb_calib_cv(t,p), 'J/K-m'))
    print(fmt.format('Cp', bhdnb.cy_Generic_hdnb_calib_cp(t,p), 'J/K-m'))
    print(fmt.format('dCpdT', hdnb.cy_Generic_hdnb_calib_dcpdt(t,p), 'J/K^2-m'))
    print(fmt.format('alpha', hdnb.cy_Generic_hdnb_calib_alpha(t,p), '1/K'))
    print(fmt.format('beta', hdnb.cy_Generic_hdnb_calib_beta(t,p), '1/bar'))
    print(fmt.format('K', hdnb.cy_Generic_hdnb_calib_K(t,p), 'bar'))
    print(fmt.format('Kp', hdnb.cy_Generic_hdnb_calib_Kp(t,p), ''))
except AttributeError:
    pass

### Available only in the "Calib" versions of generated code
Execute the parameter value/metadata functions.  
These functions are only defined for the "calibration" model code implementation:

In [ ]:
try:
    np = hdnb.cy_Generic_hdnb_get_param_number()
    names = hdnb.cy_Generic_hdnb_get_param_names()
    units = hdnb.cy_Generic_hdnb_get_param_units()
    values = hdnb.cy_Generic_hdnb_get_param_values()
    fmt = "{0:<10.10s} {1:13.6e} {2:13.6e} {3:<10.10s}"
    for i in range(0,np):
        print(fmt.format(names[i], values[i], hdnb.cy_Generic_hdnb_get_param_value(i), units[i]))
except AttributeError:
    pass

Test the functions that allow modification of the array of parameter values

In [ ]:
try:
    values[1] = 100.0
    hdnb.cy_Generic_hdnb_set_param_values(values)
    fmt = "{0:<10.10s} {1:13.6e} {2:13.6e} {3:<10.10s}"
    for i in range(0,np):
        print(fmt.format(names[i], values[i], hdnb.cy_Generic_hdnb_get_param_value(i), units[i]))
except (AttributeError, NameError):
    pass

Test the functions that allow modification of a particular parameter value

In [ ]:
try:
    hdnb.cy_Generic_hdnb_set_param_value(1, 1.0)
    fmt = "{0:<10.10s} {1:13.6e} {2:13.6e} {3:<10.10s}"
    for i in range(0,np):
        print(fmt.format(names[i], values[i], hdnb.cy_Generic_hdnb_get_param_value(i), units[i]))
except AttributeError:
    pass

Evaluate parameter derivatives ...

In [ ]:
try:
    fmt = "    {0:<10.10s} {1:13.6e}"
    for i in range(0, np):
        print ('Derivative with respect to parameter: ', names[i], ' of')
        print (fmt.format('G', hdnb.cy_Generic_hdnb_dparam_g(t, p, i)))
        print (fmt.format('dGdT', hdnb.cy_Generic_hdnb_dparam_dgdt(t, p, i)))
        print (fmt.format('dGdP', hdnb.cy_Generic_hdnb_dparam_dgdp(t, p, i)))
        print (fmt.format('d2GdT2', hdnb.cy_Generic_hdnb_dparam_d2gdt2(t, p, i)))
        print (fmt.format('d2GdTdP', hdnb.cy_Generic_hdnb_dparam_d2gdtdp(t, p, i)))
        print (fmt.format('d2GdP2', hdnb.cy_Generic_hdnb_dparam_d2gdp2(t, p, i)))
        print (fmt.format('d3GdT3', hdnb.cy_Generic_hdnb_dparam_d3gdt3(t, p, i)))
        print (fmt.format('d3GdT2dP', hdnb.cy_Generic_hdnb_dparam_d3gdt2dp(t, p, i)))
        print (fmt.format('d3GdTdP2', hdnb.cy_Generic_hdnb_dparam_d3gdtdp2(t, p, i)))
        print (fmt.format('d3GdP3', hdnb.cy_Generic_hdnb_dparam_d3gdp3(t, p, i)))
except (AttributeError, TypeError):
    pass

## Time execution of the code

In [ ]:
try:
    %timeit hdnb.cy_Generic_hdnb_calib_g(t,p)
except AttributeError:
    pass
try:
    %timeit hdnb.cy_Generic_hdnb_g(t,p)
except AttributeError:
    pass